In [5]:
import os
import numpy as np
import pandas as pd
import mne
from natsort import natsorted

In [ ]:
SAMPLE_RATE = 128  # fs
SAMPLE_LEN = 128   # T

In [6]:
root = 'EEG_data'
# Note that the eyes_open data for healthy with patient ID 5 is empty in the raw data. For simplicity, we just manually copy the eyes_closed data for this patient and paste it as eyes_open data to avoid any empty check in the code.

In [7]:
def load_selected_channels(folder_path, channel_list):
    channels_data = []
    for channel in channel_list:
        file_path = os.path.join(folder_path, f"{channel}.txt")
        
        with open(file_path, 'r') as f:
            channel_data = np.array([float(line.strip()) for line in f])
            channels_data.append(channel_data)
    
    data_array = np.stack(channels_data, axis=0).T
    return data_array

## Feature

In [8]:
# Feature
feature_path = 'Processed/ADFSU/Feature'
if not os.path.exists(feature_path):
    os.makedirs(feature_path)


pid = 1
subseq_length = SAMPLE_LEN
stride = 64  # Half of the subsequence length for half-overlapping
for file in os.listdir(root):
    print(file)
    path = os.path.join(root, file)
    eye_closed_list = []
    eye_open_list = []
    for state in os.listdir(path):
        print(state)
        state_path = os.path.join(path, state)
        subject_path_list = []
        for subject in os.listdir(state_path):
            subject_path = os.path.join(state_path, subject)
            subject_path_list.append(subject_path)
        subject_path_list = natsorted(subject_path_list)
        channel_list = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 
                'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']
        for subject_path in subject_path_list:
            data = load_selected_channels(subject_path, channel_list)
            # Calculate the number of subsequences that can be extracted
            num_subsequences = (data.shape[0] - subseq_length) // stride + 1
            # Extract the subsequences
            subsequences = [data[i * stride : i * stride + subseq_length, :] for i in range(num_subsequences)]
            features = np.array(subsequences)
            if state == 'Eyes_closed':
                eye_closed_list.append(features)
            elif state == 'Eyes_open':
                eye_open_list.append(features)
                    
    print("Eyes closed subject number:", len(eye_closed_list))
    print("Eyes open subject number:", len(eye_open_list))
    for i in range(len(eye_closed_list)):
        print(f"Patient number {i+1} eye closed array shape:{eye_closed_list[i].shape}")
        print(f"Patient number {i+1} eye open array shape:{eye_open_list[i].shape}")
        # concatenate the two states
        subject_array = np.concatenate((eye_closed_list[i], eye_open_list[i]), axis=0)

        print(f"Patient number {i+1} array shape:{subject_array.shape}\n")
        
        # Save the data
        np.save(feature_path + '/feature_{:02d}.npy'.format(pid), subject_array)
        print("Total pid is: ", pid)
        pid += 1
        

AD
Eyes_closed
Eyes_open
Eyes closed subject number: 80
Eyes open subject number: 80
Patient number 1 eye closed array shape:(15, 128, 19)
Patient number 1 eye open array shape:(15, 128, 19)
Patient number 1 array shape:(30, 128, 19)

Total pid is:  1
Patient number 2 eye closed array shape:(15, 128, 19)
Patient number 2 eye open array shape:(15, 128, 19)
Patient number 2 array shape:(30, 128, 19)

Total pid is:  2
Patient number 3 eye closed array shape:(15, 128, 19)
Patient number 3 eye open array shape:(15, 128, 19)
Patient number 3 array shape:(30, 128, 19)

Total pid is:  3
Patient number 4 eye closed array shape:(15, 128, 19)
Patient number 4 eye open array shape:(15, 128, 19)
Patient number 4 array shape:(30, 128, 19)

Total pid is:  4
Patient number 5 eye closed array shape:(15, 128, 19)
Patient number 5 eye open array shape:(15, 128, 19)
Patient number 5 array shape:(30, 128, 19)

Total pid is:  5
Patient number 6 eye closed array shape:(15, 128, 19)
Patient number 6 eye open 

In [9]:
np.load('Processed/ADFSU/Feature/feature_01.npy').shape

(30, 128, 19)

## Label

In [10]:
# Front 80 is AD, last 12 is Healthy

In [11]:
labels = np.empty((92,2), dtype='int32')
labels.shape

(92, 2)

In [12]:
for i in range(92):
    if i < 80:
        labels[i, 0] = 1
    else:
        labels[i, 0] = 0
    labels[i, 1] = i+1
print(labels)

[[ 1  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 1  5]
 [ 1  6]
 [ 1  7]
 [ 1  8]
 [ 1  9]
 [ 1 10]
 [ 1 11]
 [ 1 12]
 [ 1 13]
 [ 1 14]
 [ 1 15]
 [ 1 16]
 [ 1 17]
 [ 1 18]
 [ 1 19]
 [ 1 20]
 [ 1 21]
 [ 1 22]
 [ 1 23]
 [ 1 24]
 [ 1 25]
 [ 1 26]
 [ 1 27]
 [ 1 28]
 [ 1 29]
 [ 1 30]
 [ 1 31]
 [ 1 32]
 [ 1 33]
 [ 1 34]
 [ 1 35]
 [ 1 36]
 [ 1 37]
 [ 1 38]
 [ 1 39]
 [ 1 40]
 [ 1 41]
 [ 1 42]
 [ 1 43]
 [ 1 44]
 [ 1 45]
 [ 1 46]
 [ 1 47]
 [ 1 48]
 [ 1 49]
 [ 1 50]
 [ 1 51]
 [ 1 52]
 [ 1 53]
 [ 1 54]
 [ 1 55]
 [ 1 56]
 [ 1 57]
 [ 1 58]
 [ 1 59]
 [ 1 60]
 [ 1 61]
 [ 1 62]
 [ 1 63]
 [ 1 64]
 [ 1 65]
 [ 1 66]
 [ 1 67]
 [ 1 68]
 [ 1 69]
 [ 1 70]
 [ 1 71]
 [ 1 72]
 [ 1 73]
 [ 1 74]
 [ 1 75]
 [ 1 76]
 [ 1 77]
 [ 1 78]
 [ 1 79]
 [ 1 80]
 [ 0 81]
 [ 0 82]
 [ 0 83]
 [ 0 84]
 [ 0 85]
 [ 0 86]
 [ 0 87]
 [ 0 88]
 [ 0 89]
 [ 0 90]
 [ 0 91]
 [ 0 92]]


In [13]:
label_path = 'Processed/ADFSU/Label'
if not os.path.exists(label_path):
    os.makedirs(label_path)
np.save(label_path + '/label.npy', labels)

In [14]:
np.load('Processed/ADFSU/Label/label.npy')

array([[ 1,  1],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 1,  8],
       [ 1,  9],
       [ 1, 10],
       [ 1, 11],
       [ 1, 12],
       [ 1, 13],
       [ 1, 14],
       [ 1, 15],
       [ 1, 16],
       [ 1, 17],
       [ 1, 18],
       [ 1, 19],
       [ 1, 20],
       [ 1, 21],
       [ 1, 22],
       [ 1, 23],
       [ 1, 24],
       [ 1, 25],
       [ 1, 26],
       [ 1, 27],
       [ 1, 28],
       [ 1, 29],
       [ 1, 30],
       [ 1, 31],
       [ 1, 32],
       [ 1, 33],
       [ 1, 34],
       [ 1, 35],
       [ 1, 36],
       [ 1, 37],
       [ 1, 38],
       [ 1, 39],
       [ 1, 40],
       [ 1, 41],
       [ 1, 42],
       [ 1, 43],
       [ 1, 44],
       [ 1, 45],
       [ 1, 46],
       [ 1, 47],
       [ 1, 48],
       [ 1, 49],
       [ 1, 50],
       [ 1, 51],
       [ 1, 52],
       [ 1, 53],
       [ 1, 54],
       [ 1, 55],
       [ 1, 56],
       [ 1, 57],
       [ 1, 58],
       [ 1, 59